 	Creator Name:   Sara Baradaran, Mahdi Heidari, Zahra Khoramian 		
 	Create Date:    Aug 2020 				
 	Module Name:    Server.py 				
 	Project Name:   Secure File Sharing 	

In [1]:
import os
import socket
import hashlib
import base64
import secrets
from Crypto.Util import Counter
from Crypto import Random
import string
import json
import pandas as pd
import pyodbc
import random
from threading import Thread
from uuid import uuid4
from datetime import datetime
from Crypto.Cipher import AES
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes,serialization
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from password_strength import PasswordPolicy, PasswordStats

In [2]:

conn = pyodbc.connect('Driver={SQL Server};'
                  'Server=DESKTOP-DH5KE9Q\SQLEXPRESS;'
                  'Database=SecureFileSharing;'
                  'Trusted_Connection=yes;')

policy = PasswordPolicy.from_names(
    length=8,  # min length: 8
    uppercase=1,  # need min. 2 uppercase letters
    numbers=1,  # need min. 2 digits
    special=1,  # need min. 2 special characters
    nonletters=1,  # need min. 2 non-letter characters (digits, specials, anything)
    entropybits=30,  # need a password that has minimum 30 entropy bits (the power of its alphabet)
)

In [3]:
AouthCode = '0'

def func(connection_sock):
    
    print(str(connection_sock.getsockname()[0]))
    key = key_exchange(connection_sock)
    if key == None:
        print('Connection has been blocked. Failed to set session key')
        return 0
    while(1):
        
        global AouthCode
        data = connection_sock.recv(5000)
        if not data:
            c_IP , c_port = connection_sock.getpeername()
            print('client with ip = {} and port = {} has been disconnected at time {}'
                  .format(c_IP, c_port, str(datetime.now())))
            connection_sock.shutdown()
            connection_sock.close()
            return 1
        data = data.decode('utf-8')
        text = decrypt(data, key)
        print('received_msg', text)
        content = ''
        text = text.split("\n")
        command = text[0].split()
        
        for i in range(1, len(text) - 1):
            content += text[i]
        
        AouthCode = text[len(text) - 1]
     
        print('AouthCode', AouthCode)
        print('command', command)
        print('content', content)
        msg = ''
        
       ##############################################################################
    
        if command[0] == "register" and len(command) == 5:

            userID = check_username(command[1], conn)
            register_status = 0
            if userID == None:
                pass_str = CheckPasswordStrength(command[1], command[2])
                if pass_str == '1':
                    register_status = user_registeration(command[1], command[2], command[3], command[4], conn)
                    msg = str(register_status) + " register " + command[1] + " " + str(datetime.now())
                
                else:
                    msg = "-3 register " + command[1] + " " + str(datetime.now()) + '\n' + pass_str
            else:
                msg = "-2 register " + command[1] + " " + str(datetime.now())
            
            # register log
            add_registre_logs(command[1], command[3], command[4], str(register_status))
            
            cipher_text = encrypt(msg, key)
            if cipher_text != None:
                connection_sock.send(cipher_text.encode('utf-8'))
                
       ##############################################################################
    
        elif command[0] == "login" and len(command) == 3:
            
            ip = str(connection_sock.getsockname()[0])
            port = str(connection_sock.getsockname()[1])
            
            ban_min = check_ban(command[1], conn)
            if ban_min < 0: 
                msg = "-4 login " + command[1] + " " + str(datetime.now()) + '\n' + str(-ban_min) + '\n0'
            else:
                pass_status = check_password(command[1], command[2], conn)
                login_status = 0
                AuthCode = 0
                if pass_status:
                    login_status, AuthCode = user_login(command[1], ip, port, conn)

                    if login_status == 1:
                        msg = "1 login " + command[1] + " " + str(datetime.now()) + '\n\n' + str(AuthCode)
                    else:
                        msg = "0 login " + command[1] + " " + str(datetime.now()) + '\n\n0'
                else:
                    msg = "-1 login " + command[1] + " " + str(datetime.now()) + '\n\n0'

                #login log

                add_login_logs(command[1], command[2], ip, port, AuthCode, str(login_status))
                update_ban_state(command[1], login_status, conn)

            cipher_text = encrypt(msg, key)
            if cipher_text != None:
                connection_sock.send(cipher_text.encode('utf-8'))
                
                    
       ##############################################################################
        
        elif command[0] == "grant" and len(command) == 4:
            
            ip = str(connection_sock.getsockname()[0])
            port = str(connection_sock.getsockname()[1])
            fileID = check_file(command[1], conn)
            
            userID = check_AouthCode(ip, port, AouthCode, conn)
            owner_status = check_access(userID, fileID, 'o' , conn)
            
            userID_g = check_username(command[2], conn)
            file_conf, file_int = get_file_lables(fileID, conn)
            
            grant_status = 0
            if userID != None:
                
                if fileID != None:
                    if owner_status:
                        insert_access(userID_g, fileID, command[3], conn)
                        insert_access(userID_g, fileID, command[3], conn)
                        msg = "1 grant " + command[1] + " " + str(datetime.now())
                     
                    else:
                        msg = "0 grant " + command[1] + " " + str(datetime.now())
                else:
                    msg = "-2 grant " + command[1] + " " + str(datetime.now())
            else:
                msg = "-1 grant " + command[1] + " " + str(datetime.now())
                
                
            cipher_text = encrypt(msg, key)
            if cipher_text != None:
                connection_sock.send(cipher_text.encode('utf-8'))
                
        ##############################################################################
                
        elif command[0] == "revoce" and len(command) == 4:
            
            ip = str(connection_sock.getsockname()[0])
            port = str(connection_sock.getsockname()[1])
            fileID = check_file(command[1], conn)
            
            userID = check_AouthCode(ip, port, AouthCode, conn)
            owner_status = check_access(userID, fileID, 'o' , conn)
            
            userID_g = check_username(command[2], conn)
            file_conf, file_int = get_file_lables(fileID, conn)
            
            grant_status = 0
            if userID != None:
                
                if fileID != None:
                    if owner_status:
                        revoc_access(userID_g, fileID, command[3], conn)
                        revoc_access(userID_g, fileID, command[3], conn)
                        msg = "1 revoce " + command[1] + " " + str(datetime.now())
                     
                    else:
                        msg = "0 revoce " + command[1] + " " + str(datetime.now())
                else:
                    msg = "-2 revoce " + command[1] + " " + str(datetime.now())
            else:
                msg = "-1 revoce " + command[1] + " " + str(datetime.now())
                
                
            cipher_text = encrypt(msg, key)
            if cipher_text != None:
                connection_sock.send(cipher_text.encode('utf-8'))
        
        ##############################################################################
        elif command[0] == "put" and len(command) == 5:
            
            ip = str(connection_sock.getsockname()[0])
            port = str(connection_sock.getsockname()[1])
            creatorID = check_AouthCode(ip, port, AouthCode, conn)
            fileID = check_file(command[1], conn)

            put_status = 0
            if creatorID != None:
            
                if fileID == None:
                    put_status = put_file(command[1], content, creatorID, command[2], command[3], command[4], conn)
                    fileID = check_file(command[1], conn)
                    msg = str(put_status) + " put " + command[1] + " " + str(datetime.now())
                    if put_status:
                        insert_access(creatorID, fileID, 'w', conn)
                        insert_access(creatorID, fileID, 'r', conn)
                else:
                    msg = "-2 put " + command[1] + " " + str(datetime.now())
            else:
                msg = "-1 get " + command[1] + " " + str(datetime.now())
                
            #put log
            
            add_put_logs(creatorID, fileID, command[1], command[2], command[3], content, str(put_status))
            
            cipher_text = encrypt(msg, key)
            if cipher_text != None:
                connection_sock.send(cipher_text.encode('utf-8'))


       ##############################################################################
                
        elif command[0] == "get" and len(command) == 2:

            ip = str(connection_sock.getsockname()[0])
            port = str(connection_sock.getsockname()[1])
            userID = check_AouthCode(ip, port, AouthCode, conn)
            fileID = check_file(command[1], conn)
            owner_status = check_access(userID, fileID, 'o' , conn)
            file_conf, file_int = get_file_lables(fileID, conn)
            
            if userID != None:
                
                if fileID != None:
                    if owner_status:
                        content = get_file(command[1], conn)
                        msg = "1 get " + command[1] + " " + str(datetime.now()) + "\n" + content
                        revoc_access(userID, fileID, 'w', conn)
                        revoc_access(userID, fileID, 'r', conn)
                    else:
                        msg = "0 get " + command[1] + " " + str(datetime.now()) + "\n "
                else:
                    msg = "-2 get " + command[1] + " " + str(datetime.now()) + "\n "
            else:
                msg = "-1 get " + command[1] + " " + str(datetime.now()) + "\n "
                
            #get logs
            
            add_get_logs(userID, fileID, command[1], file_conf, file_int, str(owner_status))
                
            cipher_text = encrypt(msg, key)
            if cipher_text != None:
                connection_sock.send(cipher_text.encode('utf-8'))

       ##############################################################################

        elif command[0] == "read" and len(command) == 2:
            
            ip = str(connection_sock.getsockname()[0])
            port = str(connection_sock.getsockname()[1])
            userID = check_AouthCode(ip, port, AouthCode, conn)
            fileID = check_file(command[1], conn)
            file_conf, file_int = get_file_lables(fileID, conn)
            
            read_status = 0
            if userID != None:
                
                if fileID != None:
                    cond, BIBA_status, BLP_status, ACL = check_access_mode(userID, fileID, 'r' , conn)
                    if cond:   
                        content = read_file(command[1], conn)
                        if content == None:
                            msg = "0 1 1 read " + command[1] + " " + str(datetime.now()) + "\n "
                        else:
                            read_status = 1
                            msg = "1 1 1 read " + command[1] + " " + str(datetime.now()) + "\n" + content
                    else:
                        msg = "0 " + str(BLP_status) + " " + str(BIBA_status) + " read " + command[1] + " " + str(datetime.now()) + "\n "
                else:
                    msg = "-2 read " + command[1] + " " + str(datetime.now()) + "\n "
            else:
                msg = "-1 read " + command[1] + " " + str(datetime.now()) + "\n "
            
            #read logs
            
            add_read_logs(userID, fileID, command[1], file_conf, file_int, str(read_status))
            
            cipher_text = encrypt(msg, key)
            if cipher_text != None:
                connection_sock.send(cipher_text.encode('utf-8'))

       ##############################################################################
         
        elif command[0] == "write" and len(command) == 2:
 
            ip = str(connection_sock.getsockname()[0])
            port = str(connection_sock.getsockname()[1])
            userID = check_AouthCode(ip, port, AouthCode, conn)
            fileID = check_file(command[1], conn)
            file_conf, file_int = get_file_lables(fileID, conn)
            
            write_status = 0
            if userID != None:
                
                if fileID != None:
                    cond, BIBA_status, BLP_status, ACL = check_access_mode(userID, fileID, 'w' , conn)
                    if cond:        
                        write_status = write_file(command[1], content, conn)
                        if write_status == 0:
                            msg = "0 1 1 write " + command[1] + " " + str(datetime.now())
                        else:
                            msg = "1 1 1 write " + command[1] + " " + str(datetime.now())
                    else:
                        msg = "0 " + str(BLP_status) + " " + str(BIBA_status) + " write " + command[1] + " " + str(datetime.now())
                else:
                    msg = "-2 write " + command[1] + " " + str(datetime.now())
            else:
                msg = "-1 write " + command[1] + " " + str(datetime.now())
                
            #write logs
            add_write_logs(userID, fileID, command[1], file_conf, file_int, content, str(write_status)) 
            
            cipher_text = encrypt(msg, key)
            if cipher_text != None:
                connection_sock.send(cipher_text.encode('utf-8'))
                
                
       ##############################################################################
                
        elif command[0] == "list" and len(command) == 1:
            
            userID = check_AouthCode(ip, port, AouthCode, conn)
            ip = str(connection_sock.getsockname()[0])
            port = str(connection_sock.getsockname()[1])
            if userID == None:
                msg = "-1 list " + str(datetime.now()) 
            else: 
                msg = "1 list " + str(datetime.now()) + "\n "
                files = list_files()
                msg += files
                
            cipher_text = encrypt(msg, key)
            if cipher_text != None:
                connection_sock.send(cipher_text.encode('utf-8'))
                
                
        elif command[0] == "logout" and len(command) == 1:
            
            userID = check_AouthCode(ip, port, AouthCode, conn)
            ip = str(connection_sock.getsockname()[0])
            port = str(connection_sock.getsockname()[1])
            if userID != None:
                logout_status = user_logout(userID, ip, port, AouthCode, conn)
            else:
                msg = "-1 logout " + command[1] + " " + str(datetime.now())
                
            msg = str(logout_status) + " logout " + str(datetime.now())
            cipher_text = encrypt(msg, key)
            if cipher_text != None:
                connection_sock.send(cipher_text.encode('utf-8'))
                
        print(msg)
        


In [4]:
#Socket handling
#########################################################################################################
def setup_server():
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    if s == 0:
        print ('error in server socket creation\n')

    server_ip = socket.gethostname()
    server_port = 8500
    s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    s.bind((server_ip, server_port))
    s.listen(5)
    print("server is listening for any connection ... ")
    return s


def server_listening(s):
    connection_socket , client_addr = s.accept()
    print("client with ip = {} has been connected at time {}".format(client_addr, str(datetime.now())))
    return connection_socket


In [5]:
#Cryptography

#########################################################################################################
def key_exchange(client_sock): # first makes a private key, sends and receives public keys, then derives the secret key

    try:
        backend = default_backend()
        client_rcv_pub = client_sock.recv(200) # client_rcv_pub is client received public key in bytes
        client_pub = serialization.load_pem_public_key(client_rcv_pub, backend) # client_pub is client public key in curve object
        server_pr = ec.generate_private_key(ec.SECP256R1(), backend)
        server_pub = server_pr.public_key()
        client_sock.send(server_pub.public_bytes(serialization.Encoding.PEM, serialization.PublicFormat.SubjectPublicKeyInfo))
        shared_data = server_pr.exchange(ec.ECDH(), client_pub) 
        secret_key = HKDF(hashes.SHA256(), 32, None, b'Key Exchange', backend).derive(shared_data)
        session_key = secret_key[-16:] # to choose the last 128-bit (16-byte) of secret key
        print('key exchanged successfully.')
        return session_key
    except:
        print('error in key exchange')
        return None


def encrypt(plain_text, key): # returns a json containing ciphertext and nonce for CTR mode
    
    nonce1 = Random.get_random_bytes(8)
    countf = Counter.new(64, nonce1)
    cipher = AES.new(key, AES.MODE_CTR, counter=countf)
    cipher_text_bytes = cipher.encrypt(plain_text.encode('utf-8'))
    nonce = base64.b64encode(nonce1).decode('utf-8')
    cipher_text = base64.b64encode(cipher_text_bytes).decode('utf-8')
    result = json.dumps({'nonce':nonce, 'ciphertext':cipher_text})
    return result

def decrypt(data, key):

    try:
        b64 = json.loads(data)
        nonce = base64.b64decode(b64['nonce'].encode('utf-8'))
        cipher_text = base64.b64decode(b64['ciphertext'])
        countf = Counter.new(64, nonce)
        cipher = AES.new(key, AES.MODE_CTR, counter=countf)
        plain_text = cipher.decrypt(cipher_text)
        return plain_text.decode('utf-8')
    except ValueError:
        return None


#########################################################################################################
        
        

In [6]:
def check_access_mode(userID, fileID, access_type , conn):
    
    MyQuery = '''
    select AccessMode
    from ValidFiles where FileID = ?
    '''
    sql_query = pd.read_sql( MyQuery ,conn, params={fileID,} )
    
    acc_mode = int(str(sql_query['AccessMode']).split()[1])
    
    cond = 1
    
    BIBA = check_BIBA(userID, fileID, access_type, conn)
    BLP = check_BLP(userID, fileID, access_type, conn)
    ACL = check_access(userID, fileID, access_type, conn)
    
    if acc_mode % 2 == 0:
        cond = cond & BLP
    if acc_mode % 3 == 0:
        cond = cond & BIBA
    if acc_mode % 5 == 0:
        cond = cond & ACL
    
    return cond, BIBA, BLP, ACL


def check_AouthCode(ip, port, AouthCode, conn):
    
    MyQuery = '''
    select UserID
    from ValidConnections where [AouthCode] = ? and [Ip] = ? and [Port] = ?
    '''
    sql_query = pd.read_sql( MyQuery ,conn, params=(AouthCode, ip, port,) )
    
    if str(sql_query).split()[0] == 'Empty':
        return None
    else:
        return str(sql_query['UserID']).split()[1]


def list_files():
    
    MyQuery = '''
    select FileName, LastModifiedDate
    from ValidFiles
    '''
    sql_query = pd.read_sql( MyQuery ,conn )

    return str(sql_query)


def read_file(filename, conn):
    
    MyQuery = '''
    select Content
    from ValidFiles where FileName = ?
    '''
    sql_query = pd.read_sql( MyQuery ,conn, params={filename,} )
    if str(sql_query).split()[0] == 'Empty':
        return None
    else:
        return str(sql_query['Content'][0])


def write_file(filename, content, conn):
    
    MyQuery = '''
    update Files set Content = ?
    where [FileName] = ? and [Status] = '1'
    '''
    cursor = conn.cursor()
    cursor.execute( MyQuery, content, filename)
    conn.commit()
    cursor.close()
    
    new_content = read_file(filename, conn)
    if content == new_content:
        return 1
    return 0
        
    
def get_file(filename, conn):
    
    content = read_file(filename, conn)
    
    MyQuery = '''
    update Files set [Status] = '0'
    where [FileName] = ? and [Status] = '1'
    '''
    cursor = conn.cursor()
    cursor.execute( MyQuery, filename)
    conn.commit()
    cursor.close()
    
    return content
    


In [7]:

def check_username(username, conn):

    MyQuery = '''
    select UserID
    from Users where UserName = ?
    '''
    sql_query = pd.read_sql( MyQuery ,conn, params={username,} )
    
    if str(sql_query).split()[0] == 'Empty':
        return None
    else:
        return str(sql_query['UserID']).split()[1]
            
#conf_range = ['Top Secret', 'Secret', 'Confidential', 'Unclassified']
#int_range = ['Very Trusted', 'Trusted', 'Slightly Trusted', 'UnTrusted']

int_range = ['1', '2', '3', '4']
conf_range = ['1', '2', '3', '4']

def user_registeration(username, password, conf_label, integrity_label, conn):
    if username != "all":
        
        salt = ''.join(secrets.choice(string.ascii_letters) for _ in range(25))
        pass_hash = hashlib.sha256((password + salt).encode('utf-8')).hexdigest()
        pass_hash = str(pass_hash)

        MyQuery = '''
        insert into Users ( UserName, PasswordHash, Salt, ConfLable, IntegrityLable) values (?, ?, ?, ?, ? );
        '''

        if conf_label in conf_range and integrity_label in int_range:
            cursor = conn.cursor()
            cursor.execute( MyQuery, username, pass_hash,salt, conf_label, integrity_label)
            conn.commit()
            cursor.close()
            if check_username(username, conn) == None:
                return 0
            return 1
    return 0


In [8]:

def CheckPasswordStrength(username, password):
    
    if password.find(username) != -1:
        return "password should not include username"
    #check condition 1
    Condition1 = policy.test(password)
    if Condition1:
        return ''.join(str(Condition1))
    #check condition 2
    f = open("PwnedPasswordsTop100k.txt","r")
    for x in f:
        x = str(x).strip()
        if x == password:
            return "Password is in top 100,000 pwned passwords"
    
    return '1'

def check_password(username, password, conn):
    
    MyQuery1 = '''
    select [Salt]
    from Users where UserName = ?
    '''
    MyQuery2 = '''
    select dbo.CheckPassHash(?, ?) as correctness
    '''
    sql_query = pd.read_sql( MyQuery1 ,conn, params={username,} )
    
    salt = str(sql_query['Salt']).split()[1]
    print('salt',salt)
    pass_h = str(hashlib.sha256((password + salt).encode('utf-8')).hexdigest())
    print('pass_h',pass_h)
    print(username)
    sql_query2 = pd.read_sql( MyQuery2 ,conn, params=(username, pass_h) )
    print(sql_query2)
    if str(sql_query2['correctness']).split()[1] == '1':
        return 1
    else:
        return 0


In [9]:

def check_file(filename, conn):

    MyQuery = '''
    select FileID
    from ValidFiles where FileName = ?
    '''
    sql_query = pd.read_sql( MyQuery ,conn, params={filename,} )
    
    if str(sql_query).split()[0] == 'Empty':
        return None
    else:
        return str(sql_query['FileID']).split()[1]

    
def put_file(filename, content, creatorID, conf_label, integrity_label, access_mode , conn):
    
    MyQuery = '''
    insert into Files ([FileName], [FileCreatorID], [ConfLable], [IntegrityLable], [AccessMode], [Content], [Status])
    values (?, ?, ?, ?, ?, ?, ?)
    '''
    cursor = conn.cursor()
    cursor.execute(MyQuery, filename, creatorID, conf_label, integrity_label, access_mode, content, '1')
    conn.commit()
    cursor.close()
    
    if check_file(filename, conn) == None:
        return 0
    return 1


def update_ban_state(username, status, conn):
        
    MyQuery1 = '''UPDATE BanUser
    SET StartBanTime = CURRENT_TIMESTAMP, BanLvl= ?
    where UserID = (select UserID from Users where UserName= ?)
    '''
    cursor = conn.cursor()
    if status:
        cursor.execute(MyQuery1, 0, username)
        
    else:      
        MyQuery2 = '''
        select dbo.FindLastFailedLogin(?) as lastfail
        '''
        sql_query2 = pd.read_sql( MyQuery2 ,conn, params={username,} ) 
        lastfail = str(sql_query2['lastfail']).split()[1]
        lastfail = int(lastfail)
        if lastfail % 3 == 0:
            cursor.execute(MyQuery1, lastfail//3, username)
        
    conn.commit()
    cursor.close()

def check_ban(username, conn):
    
    MyQuery2 = '''
    select dbo.IsBan(?) as ban_min
    '''
    sql_query2 = pd.read_sql( MyQuery2 ,conn, params={username,} ) 
    return int(str(sql_query2['ban_min']).split()[1])



In [10]:

#Logs

##############################################################################
def add_registre_logs(username, conf_lable, integrity_lable, status):
    
    MyQuery = '''
    INSERT INTO RegisterLogs ( UserName, ConfLable, IntegrityLable, [Status])
    VALUES (?, ?, ?, ?); 
    '''
    cursor = conn.cursor()
    cursor.execute(MyQuery, username, conf_lable, integrity_lable, status)
    conn.commit()
    cursor.close()
    
def add_login_logs(username, password, ip, port, AouthCode, status):

    MyQuery = '''
    INSERT INTO LoginLogs ( UserName, [password], ConnectionIp, ConnectionPort, AuthenticationCode, [Status])
    VALUES (?, ?, ?, ?, ?, ?);
    '''
    cursor = conn.cursor()
    cursor.execute(MyQuery, username, password, ip, port, AouthCode, status)
    conn.commit()
    cursor.close()
    
def add_put_logs(creatorID, fileID, filename, file_conf, file_int, content, status):
    
    MyQuery = '''
    INSERT INTO PutLogs ( CreatorID, FileID, FileName ,CurFileConfLable, CurFileIntegrityLable, Content, [Status])
    VALUES (?, ?, ?, ?, ?, ?, ?);
    '''
    if fileID == None:
        fileID = 0
    cursor = conn.cursor()
    cursor.execute(MyQuery, creatorID, fileID, filename, file_conf, file_int, content, status)
    conn.commit()
    cursor.close()

def add_get_logs(userID, fileID, filename, file_conf, file_int, status):

    MyQuery = '''
    INSERT INTO GetLogs ( UserID, FileID, FileName ,CurFileConfLable, CurFileIntegrityLable, [Status])
    VALUES (?, ?, ?, ?, ?, ?);
    '''
    if fileID == None:
        fileID = 0
    
    cursor = conn.cursor()
    cursor.execute(MyQuery, userID, fileID, filename, file_conf, file_int, status)
    conn.commit()
    cursor.close()

def add_read_logs(userID, fileID, filename, file_conf, file_int, status):

    MyQuery = '''
    INSERT INTO ReadLogs ( UserID, FileID, FileName, CurFileConfLable, CurFileIntegrityLable, [Status])
    VALUES (?, ?, ?, ?, ?, ?);
    '''
    if fileID == None:
        fileID = 0
        
    print(userID, fileID, filename, file_conf, file_int, status)
    cursor = conn.cursor()
    cursor.execute(MyQuery, userID, fileID, filename, file_conf, file_int, status)
    conn.commit()
    cursor.close()
    
def add_write_logs(userID, fileID, filename, file_conf, file_int, content, status):

    MyQuery = '''
    INSERT INTO WriteLogs ( UserID, FileID, FileName, CurFileConfLable, CurFileIntegrityLable, Content, [Status])
    VALUES (?, ?, ?, ?, ?, ?, ?);
    '''
    if fileID == None:
        fileID = 0
        
    cursor = conn.cursor()
    cursor.execute(MyQuery, userID, fileID, filename, file_conf, file_int, content, status)
    conn.commit()
    cursor.close()
 
 ##############################################################################   
    

In [11]:

#Access Control

##############################################################################
def insert_access(userID, fileID, access_type, conn):
    
    MyQuery = '''
    insert into AccessList ([UserID], [FileID], [AccessType])
    values (?, ?, ?)
    '''
    
    if check_access(userID, fileID, access_type, conn) == 0:
    
        cursor = conn.cursor()
        cursor.execute(MyQuery, userID, fileID, access_type)
        conn.commit()
        cursor.close()
    

def revoc_access(userID, fileID, access_type, conn):

    MyQuery = '''
    delete AccessList
    where UserID = ? and FileID = ? and AccessType = ?
    '''
    cursor = conn.cursor()
    cursor.execute(MyQuery, userID, fileID, access_type)
    conn.commit()
    cursor.close()

def check_access(userID, fileID, access_type, conn):
    
    if access_type == 'o':
        MyQuery = '''
        select *
        from Files
        where FileID = ? and FileCreatorID = ? 
        '''
        sql_query = pd.read_sql( MyQuery ,conn, params=(fileID, userID,) )
    else:
        
        MyQuery = '''
        select *
        from AccessList
        where FileID = ? and UserID = ? and AccessType = ? 
        '''
        sql_query = pd.read_sql( MyQuery ,conn, params=(fileID, userID, access_type,) )
    
    if str(sql_query).split()[0] == 'Empty':
        return 0
    else:
        return 1

def get_file_lables(fileID, conn):
    
    MyQuery = '''
    select[IntegrityLable], [ConfLable]
    from Files
    where FileID = ?
    '''
    sql_query = pd.read_sql( MyQuery ,conn, params={fileID,} ) 
    

    if str(sql_query).split()[0] == 'Empty':
        file_conf = ''
        file_int = ''
        
    else:
        file_conf = str(sql_query['ConfLable']).split()[1]
        file_int = str(sql_query['IntegrityLable']).split()[1]
    
    return file_conf, file_int
    

def check_BLP(userID, fileID, access_type, conn):
    
    
    MyQuery1 = '''
    select[ConfLable]
    from Users
    where UserID = ?
    '''
    sql_query1 = pd.read_sql( MyQuery1 ,conn, params={userID,} )
    
    MyQuery2 = '''
    select[ConfLable]
    from Files
    where FileID = ?
    '''
    sql_query2 = pd.read_sql( MyQuery2 ,conn, params={fileID,} )
    
    user_conf = int(str(sql_query1['ConfLable']).split()[1])
    file_conf = int(str(sql_query2['ConfLable']).split()[1])
    
    if access_type == 'w':
        if(user_conf > file_conf):
            return 0
        else:
            return 1
    
    elif access_type == 'r':
        if(user_conf < file_conf):
            return 0
        else:
            return 1
     
    
def check_BIBA(userID, fileID, access_type, conn):
    
    
    MyQuery1 = '''
    select[IntegrityLable]
    from Users
    where UserID = ?
    '''
    sql_query1 = pd.read_sql( MyQuery1 ,conn, params={userID,} )
    
    MyQuery2 = '''
    select[IntegrityLable]
    from Files
    where FileID = ?
    '''
    sql_query2 = pd.read_sql( MyQuery2 ,conn, params={fileID,} )
    
    user_int = int(str(sql_query1['IntegrityLable']).split()[1])
    file_int = int(str(sql_query2['IntegrityLable']).split()[1])
    
    if access_type == 'w':
        if(user_int < file_int):
            return 0
        else:
            return 1
    
    elif access_type == 'r':
        if(user_int > file_int):
            return 0
        else:
            return 1

##############################################################################

In [12]:

def user_login(username, Ip, Port, conn):
        
    userID = check_username(username, conn)
    
    MyQuery = '''
    insert into Connections ([UserID], [Ip], [Port], [AouthCode], [ConnectionDate], [Status])
    values (?, ?, ?, ?, ?, ?)
    '''
    AouthCode = uuid4()
    ConnectionDate = str(datetime.now())[:-3]
    
    cursor = conn.cursor()
    cursor.execute( MyQuery, userID, Ip, Port, AouthCode, ConnectionDate, '1')
    conn.commit()
    cursor.close()
    
    MyQuery2 = '''
    select  [UserID], [Ip], [Port], [AouthCode], [ConnectionDate], [Status]
    from connections
    where [UserID] = ? and [Ip] = ? and [Port] = ? and [AouthCode] = ? and [ConnectionDate] = ? and [Status] = '1' 
    '''
    
    sql_query2 = pd.read_sql( MyQuery2 ,conn, params=(userID, Ip, Port, AouthCode, ConnectionDate,) )

    if str(sql_query2).split()[0] == 'Empty':
        return 0 , None
    
    return 1, AouthCode

def user_logout(userID, Ip, Port, AouthCode, conn):
        
    MyQuery = '''
    update connections set [Status] = '0', ConnectionCloseDate = ? 
    where [UserID] = ? and [Ip] = ? and [Port] = ? and [AouthCode] = ? and [Status] = '1'
    '''
    
    ConnectionCloseDate = str(datetime.now())
    
    cursor = conn.cursor()
    cursor.execute( MyQuery, ConnectionCloseDate, userID, Ip, Port, AouthCode)
    conn.commit()
    cursor.close()
    
    MyQuery2 = '''
    select [Status]
    from connections
    where [UserID] = ? and [Ip] = ? and [Port] = ? and [AouthCode] = ? and [Status] = '0' and ConnectionCloseDate = ?
    '''
    sql_query2 = pd.read_sql( MyQuery2 ,conn, params=(userID, Ip, Port, AouthCode, ConnectionCloseDate) )

    if str(sql_query2).split()[0] == 'Empty':
        return 0
    return 1


In [ ]:

if __name__ == "__main__":
    
    s = setup_server()
    while True:
        connection_sock = server_listening(s)
        try:
            Thread(target=func,args=(connection_sock,)).start()
        except:
            print('Server is busy. Unable to create more threads.')
    s.shutdown()
    s.close()

server is listening for any connection ... 
client with ip = ('192.168.56.1', 13420) has been connected at time 2021-05-16 09:15:20.098786
192.168.56.1
key exchanged successfully.
